In [1]:
import pandas

In [2]:
import pandas as pd
from bs4 import BeautifulSoup

with open('data/nt_full_060124.html', 'r', encoding='utf-8') as f:

    html_content = f.read()

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract rows
    table_content = soup.select_one('tbody.table-content')
    # Extracting data for each player
data = []
for row in table_content.select('tr.hasCaptainsCol'):

    name = row.select_one('td.name-cell span').text
    position = row.select_one('td.position-cell ft-tooltip')['text']
    team = row.select_one('td.team-cell ft-tooltip')['text']
    price = row.select_one('td.price-cell').text
    ownership_pct = row.select_one('td.selectedBy-cell span.value').text
    captain_pct = row.select_one('td.captainedBy-cell span.value').text
    points = row.select_one('td.points-cell ft-bubble span').text
    
    data.append([name, position, team, price, ownership_pct, captain_pct, points])

    
# Convert data to a pandas DataFrame
norsk_tipping = pd.DataFrame(data)

print(norsk_tipping)


                    0         1                  2      3       4       5  \
0           C. Palmer  Midtbane            Chelsea   5.5M  38.55%   0.35%   
1            M. Salah  Midtbane          Liverpool  12.5M  62.38%  26.11%   
2            M. Olise  Midtbane     Crystal Palace   6.0M   0.72%   0.03%   
3           M. Kilman   Forsvar             Wolves   4.3M   0.87%       -   
4           C. Dawson   Forsvar             Wolves   4.2M   1.01%       -   
..                ...       ...                ...    ...     ...     ...   
530      L. Vigouroux    Keeper            Burnley   4.0M   0.03%       -   
531       R. Matthews    Keeper     Crystal Palace   4.0M   0.02%       -   
532  B. Baker-Boaitey  Midtbane           Brighton   4.5M       -       -   
533           S. Peck  Midtbane   Sheffield United   4.5M       -       -   
534       W. Kambwala   Forsvar  Manchester United   4.5M       -       -   

         6  
0    15.70  
1    15.10  
2    13.70  
3    12.30  
4    12.30

In [3]:
norsk_tipping.columns = ["Name", "Position", "Team", "Price", "Ownership", "Captain%", "PointsRound"]

In [4]:
norsk_tipping["Price"] = norsk_tipping["Price"].str.extract("^[+-]?([0-9]+([.][0-9]*)?|[.][0-9]+)M")[0]
norsk_tipping["Price"] = pd.to_numeric(norsk_tipping["Price"])

In [5]:
norsk_tipping.sort_values("Price", inplace=True)

In [6]:
norsk_tipping["Surname"] = norsk_tipping["Name"].str.rsplit(". ", n=1, expand=True).fillna("").apply(lambda x : max(x,key=len),axis=1)

In [7]:
norsk_tipping

,Name,Position,Team,Price,Ownership,Captain%,PointsRound,Surname
267,H. Delcroix,Forsvar,Burnley,4.0,-,-,0.70,Delcroix
477,J. Shea,Keeper,Luton,4.0,2.92%,-,-,Shea
463,H. Ekdal,Forsvar,Burnley,4.0,0.47%,-,-,Ekdal
461,J. Amissah,Keeper,Sheffield United,4.0,0.17%,-,-,Amissah
459,A. A. Davies,Keeper,Sheffield United,4.0,0.21%,-,-,Davies
...,...,...,...,...,...,...,...,...
11,H. Son,Midtbane,Spurs,9.3,51.01%,8.22%,8.70,Son
301,K. De Bruyne,Midtbane,Manchester City,10.0,2.45%,0.47%,-,De Bruyne
16,B. Saka,Midtbane,Arsenal,10.2,39.27%,2.06%,7.70,Saka
1,M. Salah,Midtbane,Liverpool,12.5,62.38%,26.11%,15.10,Salah


## Get values from FPL Review

In [8]:
fplreview = pd.read_csv("confidential_data/fplreview_1704556496.csv")

point_columns = [ col for col in fplreview.columns if "_Pts" in col]

point_columns

fplreview["Total Points"] = fplreview[point_columns].sum(axis=1)

fplreview.sort_values("Total Points",ascending=False, inplace=True)
fplreview

,Pos,ID,Name,BV,SV,Team,21_xMins,21_Pts,22_xMins,22_Pts,...,29_xMins,29_Pts,30_xMins,30_Pts,31_xMins,31_Pts,32_xMins,32_Pts,Elite%,Total Points
354,F,355,Haaland,13.9,13.9,Man City,55,4.17,86,7.56,...,79,5.29,78,4.71,80,6.05,80,5.17,0.03,73.52
18,M,19,Saka,9.0,9.0,Arsenal,89,5.79,85,5.16,...,78,5.17,79,3.24,78,6.50,77,4.35,0.92,61.64
84,F,85,Solanke,7.1,6.9,Bournemouth,90,4.35,87,4.37,...,81,4.23,81,4.83,80,4.70,82,5.12,0.80,61.13
59,F,60,Watkins,8.9,8.6,Aston Villa,90,4.72,87,5.16,...,84,4.69,83,5.19,84,3.22,84,5.63,0.99,60.95
13,M,14,Ødegaard,8.4,8.4,Arsenal,87,5.22,85,4.88,...,81,4.94,81,3.17,78,5.89,79,4.17,0.04,58.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,D,305,Matip,4.9,4.9,Liverpool,0,0.00,0,0.00,...,0,0.00,0,0.00,0,0.00,0,0.00,0.00,0.00
298,M,299,Henderson,5.0,5.0,Liverpool,0,0.00,0,0.00,...,0,0.00,0,0.00,0,0.00,0,0.00,0.00,0.00
295,M,296,Fabinho,5.0,5.0,Liverpool,0,0.00,0,0.00,...,0,0.00,0,0.00,0,0.00,0,0.00,0.00,0.00
288,G,289,Adrián,3.9,3.9,Liverpool,0,0.00,0,0.00,...,0,0.00,0,0.00,0,0.00,0,0.00,0.00,0.00


In [9]:
norsk_tipping["Surname"].isin(fplreview["Name"].to_list()).value_counts()

Surname
True     422
False    113
Name: count, dtype: int64

In [10]:
from unidecode import unidecode

fplreview["Name"] = fplreview["Name"].str.rsplit(".", n=1, expand=True).fillna("").apply(lambda x : max(x,key=len),axis=1).apply(unidecode)
norsk_tipping["Surname"] = norsk_tipping["Surname"].apply(unidecode)

In [11]:
mapper = {"Man City": "Manchester City",
          "Man Utd": "Manchester United",
          "Nott'm Forest": "Nottingham Forest",
          "Sheffield Utd": "Sheffield United",
         }

In [12]:
fplreview["Team"] = fplreview.Team.replace(mapper)

## Replace any name that is mismatched

In [13]:
liverpool = {"Nunez": "Darwin", 
             "Jota": "Diogo J", 
             "Diaz": "Luis Diaz", 
             "Alisson": "Becker", 
             "van Dijk": "Virgil", 
             "Alcantara": "Thiago"}

man_city = {"Silva": "Bernardo", 
            "Rodri":"Rodrigo", 
            "Ederson": "Ederson M", 
            "Dias": "Ruben",
            "Ortega": "Ortega Moreno",
            "Gomez": "Sergio Gomez",
            "Nunes": "Matheus N"}

aston_villa = {"Luiz": "Douglas Luiz",
                "Traore": "Bertrand Traore",
                "Torres": "Pau",
                "Moreno": "Alex Moreno",
                # "Konsa": "???",
                "Carlos": "Diego Carlos"}

wolves = {"Hee-chan": "Hee Chan",
          "Silva": "Fabio Silva",
          "Ait Nouri": "Ait-Nouri",
          'Sa': "Jose Sa",
          'Antonio Gomes': "Toti",
          #'H.Bueno',
          #'S.Bueno',
          'Traore': "Boubacar Traore",
          #'Sanderson': " No entry",
          'Castro': "Otto",
          'Gomes': "Joao Gomes"}

fulham = { 'Adarabioyo': "Tosin", 
          'De-Cordova Reid': "De Cordova-Reid", 
          'Traore': "Adama", 
          'Jimenez': "Raul", 
          'Pereira': "Andreas"}

brighton = {' van Hecke': "Van Hecke" , 
            #'Sima':"No entry", 
            'Pedro': "Joao Pedro", 
            'Fati': "Ansu Fati"}

spurs = {
         'Emerson': "Royal", 
          'van de Ven': "Van de Ven", 
          'Gil': "Bryan", 
          'Porro': "Pedro Porro", 
          'Bentacur': "Bentancur"
}

chelsea = {
    'Fernandez': "Enzo", 
    'Washington': "Deivid"
}
unique = {"Guimaraes": "Bruno G",
          'van de Beek': "Van de Beek", 
          'Bayindir': "Altay", 
          'Mejbri': "Hannibal"
          }

In [14]:
norsk_tipping.loc[norsk_tipping.Team=="Liverpool", "Surname"] = norsk_tipping.loc[norsk_tipping.Team=="Liverpool", "Surname"].replace(liverpool)
norsk_tipping.loc[norsk_tipping.Team=="Manchester City","Surname"] = norsk_tipping.loc[norsk_tipping.Team=="Manchester City","Surname"].replace(man_city)
norsk_tipping.loc[norsk_tipping.Team=="Aston Villa", "Surname"] = norsk_tipping.loc[norsk_tipping.Team=="Aston Villa", "Surname"].replace(aston_villa)
norsk_tipping.loc[norsk_tipping.Team=="Wolves", "Surname"] = norsk_tipping.loc[norsk_tipping.Team=="Wolves", "Surname"].replace(wolves)
norsk_tipping.loc[norsk_tipping.Team=="Fulham", "Surname"] = norsk_tipping.loc[norsk_tipping.Team=="Fulham", "Surname"].replace(fulham)
norsk_tipping.loc[norsk_tipping.Team=="Brighton", "Surname"] = norsk_tipping.loc[norsk_tipping.Team=="Brighton", "Surname"].replace(brighton)
norsk_tipping.loc[norsk_tipping.Team=="Spurs", "Surname"] = norsk_tipping.loc[norsk_tipping.Team=="Spurs", "Surname"].replace(spurs)
norsk_tipping.loc[norsk_tipping.Team=="Chelsea", "Surname"] = norsk_tipping.loc[norsk_tipping.Team=="Chelsea", "Surname"].replace(chelsea)


norsk_tipping["Surname"] = norsk_tipping["Surname"].replace(unique)

In [15]:
df = norsk_tipping.merge(fplreview,left_on=["Team", "Surname"], right_on=["Team", "Name"], how="left",suffixes=("_nt", "_review"))

In [16]:
df[df.Pos.isna()]#.Team.value_counts()
                

,Name_nt,Position,Team,Price,Ownership,Captain%,PointsRound,Surname,Pos,ID,...,29_xMins,29_Pts,30_xMins,30_Pts,31_xMins,31_Pts,32_xMins,32_Pts,Elite%,Total Points
16,F. Stevens,Forsvar,Brentford,4.0,0.09%,-,-,Stevens,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,N. Ferguson,Forsvar,Crystal Palace,4.0,0.12%,-,-,Ferguson,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,M. Roerslev Rasmussen,Forsvar,Brentford,4.3,-,-,3.70,Roerslev Rasmussen,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,H. H.Bueno,Forsvar,Wolves,4.3,0.05%,-,1.00,H.Bueno,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,S. S.Bueno,Forsvar,Wolves,4.4,-,-,-,S.Bueno,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,O. Vlachodimos,Keeper,Nottingham Forest,4.5,-,-,-,Vlachodimos,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,A. Gomes,Midtbane,Everton,4.5,0.09%,-,1.00,Gomes,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,E. Konsa,Forsvar,Aston Villa,4.5,3.69%,-,0.30,Konsa,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,D. Sanderson,Forsvar,Wolves,4.5,0.02%,-,-,Sanderson,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,A. Santos,Midtbane,Nottingham Forest,4.5,0.07%,-,-,Santos,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df[(df.Pos.isna()) & (df.Team == "Manchester United")].Surname.to_list()
                

[]

In [18]:
fplreview[fplreview.Team.isin(["Manchester United"])].head(50).sort_values("Name")

,Pos,ID,Name,BV,SV,Team,21_xMins,21_Pts,22_xMins,22_Pts,...,29_xMins,29_Pts,30_xMins,30_Pts,31_xMins,31_Pts,32_xMins,32_Pts,Elite%,Total Points
369,D,370,Alex Telles,4.5,4.5,Manchester United,0,0.00,0,0.00,...,0,0.00,0,0.00,0,0.00,0,0.00,0.00,0.00
693,G,695,Altay,4.0,4.0,Manchester United,93,3.46,79,2.99,...,4,0.18,4,0.16,4,0.13,4,0.15,0.00,10.10
370,M,371,Amad,4.4,4.4,Manchester United,7,0.43,5,0.27,...,9,0.63,9,0.50,9,0.47,8,0.43,0.00,5.29
707,M,709,Amrabat,5.0,5.0,Manchester United,0,0.00,0,0.00,...,15,0.58,13,0.48,13,0.45,14,0.47,0.00,4.17
371,M,372,Antony,6.8,6.8,Manchester United,40,2.11,40,1.91,...,39,2.42,39,1.95,40,1.81,41,1.82,0.00,23.54
374,D,375,Bailly,4.0,4.0,Manchester United,0,0.00,0,0.00,...,0,0.00,0,0.00,0,0.00,0,0.00,0.00,0.00
766,D,768,Bennett,4.0,4.0,Manchester United,0,0.00,0,0.00,...,0,0.00,0,0.00,0,0.00,0,0.00,0.00,0.00
375,M,376,Casemiro,5.3,5.3,Manchester United,51,1.74,86,2.64,...,82,3.25,82,2.57,83,2.38,82,2.35,0.00,30.20
376,D,377,Dalot,5.1,5.1,Manchester United,63,2.07,52,1.72,...,53,2.80,50,1.78,50,1.26,52,1.29,0.01,20.91
378,M,379,Eriksen,5.8,5.8,Manchester United,43,2.12,39,1.79,...,38,2.25,38,1.78,37,1.62,37,1.58,0.00,21.33


In [26]:
df.sort_values("Total Points",ascending=False).head(50)
                

,Name_nt,Position,Team,Price,Ownership,Captain%,PointsRound,Surname,Pos,ID,...,29_xMins,29_Pts,30_xMins,30_Pts,31_xMins,31_Pts,32_xMins,32_Pts,Elite%,Total Points
538,E. Haaland,Angrep,Manchester City,12.9,45.97%,25.41%,-,Haaland,F,355.0,...,79.0,5.29,78.0,4.71,80.0,6.05,80.0,5.17,0.03,73.52
536,B. Saka,Midtbane,Arsenal,10.2,39.27%,2.06%,7.70,Saka,M,19.0,...,78.0,5.17,79.0,3.24,78.0,6.50,77.0,4.35,0.92,61.64
498,D. Solanke,Angrep,Bournemouth,6.9,39.91%,1.47%,3.50,Solanke,F,85.0,...,81.0,4.23,81.0,4.83,80.0,4.70,82.0,5.12,0.80,61.13
526,O. Watkins,Angrep,Aston Villa,8.4,61.68%,8.73%,9.70,Watkins,F,60.0,...,84.0,4.69,83.0,5.19,84.0,3.22,84.0,5.63,0.99,60.95
532,M. Ødegaard,Midtbane,Arsenal,9.1,17.11%,1.94%,2.70,Odegaard,M,14.0,...,81.0,4.94,81.0,3.17,78.0,5.89,79.0,4.17,0.04,58.34
537,M. Salah,Midtbane,Liverpool,12.5,62.38%,26.11%,15.10,Salah,M,308.0,...,80.0,5.31,79.0,6.16,79.0,8.05,79.0,5.25,0.92,54.39
502,T. Alexander-Arnold,Forsvar,Liverpool,7.0,46.60%,0.96%,0.30,Alexander-Arnold,D,290.0,...,80.0,3.81,79.0,4.16,78.0,5.84,79.0,3.57,0.80,51.19
533,B. Fernandes,Midtbane,Manchester United,9.3,11.51%,1.12%,2.70,Fernandes,M,373.0,...,77.0,5.57,78.0,4.10,77.0,3.57,76.0,3.40,0.03,50.91
511,Richarlison,Angrep,Spurs,7.4,7.10%,0.33%,6.70,Richarlison,M,509.0,...,60.0,3.65,60.0,5.06,60.0,3.72,58.0,4.50,0.23,50.85
535,K. De Bruyne,Midtbane,Manchester City,10.0,2.45%,0.47%,-,De Bruyne,M,349.0,...,56.0,3.45,56.0,3.20,56.0,3.84,55.0,3.40,0.00,50.13


In [20]:
df

,Name_nt,Position,Team,Price,Ownership,Captain%,PointsRound,Surname,Pos,ID,...,29_xMins,29_Pts,30_xMins,30_Pts,31_xMins,31_Pts,32_xMins,32_Pts,Elite%,Total Points
0,H. Delcroix,Forsvar,Burnley,4.0,-,-,0.70,Delcroix,D,674.0,...,25.0,0.91,26.0,0.54,26.0,0.90,28.0,0.72,0.00,7.66
1,J. Shea,Keeper,Luton,4.0,2.92%,-,-,Shea,G,335.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.00
2,H. Ekdal,Forsvar,Burnley,4.0,0.47%,-,-,Ekdal,D,167.0,...,25.0,0.80,28.0,0.48,28.0,0.87,28.0,0.62,0.00,6.34
3,J. Amissah,Keeper,Sheffield United,4.0,0.17%,-,-,Amissah,G,472.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.00
4,A. A. Davies,Keeper,Sheffield United,4.0,0.21%,-,-,Davies,M,666.0,...,20.0,0.75,20.0,0.79,20.0,0.71,20.0,0.77,0.00,8.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,H. Son,Midtbane,Spurs,9.3,51.01%,8.22%,8.70,Son,M,516.0,...,84.0,5.17,85.0,7.53,86.0,5.38,85.0,6.85,0.81,47.64
535,K. De Bruyne,Midtbane,Manchester City,10.0,2.45%,0.47%,-,De Bruyne,M,349.0,...,56.0,3.45,56.0,3.20,56.0,3.84,55.0,3.40,0.00,50.13
536,B. Saka,Midtbane,Arsenal,10.2,39.27%,2.06%,7.70,Saka,M,19.0,...,78.0,5.17,79.0,3.24,78.0,6.50,77.0,4.35,0.92,61.64
537,M. Salah,Midtbane,Liverpool,12.5,62.38%,26.11%,15.10,Salah,M,308.0,...,80.0,5.31,79.0,6.16,79.0,8.05,79.0,5.25,0.92,54.39
